In [1]:
import pandas as pd
import defs
import requests

In [8]:
sessions = requests.Session()

In [3]:
instrument_df = pd.read_pickle('instruments.pkl')

In [4]:
currencies = ['EUR', 'USD', 'AUD', 'GBP', 'NZD', 'CAD']

In [5]:
for pair1 in currencies:
    for pair2 in currencies:            
        pair = f'{pair1}_{pair2}'
        if pair in instrument_df.name.unique():
            print(pair)
    

EUR_USD
EUR_AUD
EUR_GBP
EUR_NZD
EUR_CAD
USD_CAD
AUD_USD
AUD_NZD
AUD_CAD
GBP_USD
GBP_AUD
GBP_NZD
GBP_CAD
NZD_USD
NZD_CAD


In [11]:
def fetch_candles(pair_name, count, granularity):
    url = f'{defs.OANDA_URL}/instruments/{pair_name}/candles'
    params = dict(
        count = count,
        granularity = granularity,
        price = 'MBA'
    )
    response = sessions.get(url, params=params, headers=defs.SECURE_HEADER)
    return response.status_code, response.json()

In [21]:
code, res = fetch_candles('EUR_USD', 10, 'H1')

In [32]:
def get_candles_df(json_response):
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']

    my_data = []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']        
        for price in prices:
            for oh in ohlc:
                new_dict[f'{price}_{oh}'] = candle[price][oh]
        my_data.append(new_dict)
    return pd.DataFrame.from_dict(my_data)


In [33]:
df = get_candles_df(res)

In [34]:
df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-02-12T22:00:00.000000000Z,680,1.07738,1.07760,1.07706,1.07728,1.07721,1.07743,1.07685,1.07718,1.07755,1.07777,1.07726,1.07737
1,2024-02-12T23:00:00.000000000Z,892,1.07730,1.07730,1.07708,1.07720,1.07717,1.07721,1.07692,1.07713,1.07744,1.07748,1.07716,1.07727
2,2024-02-13T00:00:00.000000000Z,1291,1.07718,1.07733,1.07670,1.07730,1.07711,1.07725,1.07663,1.07722,1.07726,1.07741,1.07678,1.07737
3,2024-02-13T01:00:00.000000000Z,1314,1.07731,1.07746,1.07656,1.07670,1.07724,1.07739,1.07649,1.07662,1.07738,1.07753,1.07664,1.07677
4,2024-02-13T02:00:00.000000000Z,1371,1.07668,1.07670,1.07622,1.07643,1.07660,1.07661,1.07614,1.07636,1.07676,1.07678,1.07630,1.07650
5,2024-02-13T03:00:00.000000000Z,1331,1.07642,1.07648,1.07606,1.07618,1.07635,1.07640,1.07598,1.07610,1.07650,1.07656,1.07614,1.07625
6,2024-02-13T04:00:00.000000000Z,766,1.07619,1.07644,1.07613,1.07638,1.07611,1.07636,1.07605,1.07630,1.07627,1.07651,1.07621,1.07646
7,2024-02-13T05:00:00.000000000Z,868,1.07634,1.07680,1.07634,1.07670,1.07626,1.07672,1.07626,1.07663,1.07642,1.07688,1.07642,1.07677
8,2024-02-13T06:00:00.000000000Z,1304,1.07669,1.07726,1.07663,1.07704,1.07661,1.07719,1.07656,1.07697,1.07677,1.07734,1.07670,1.07712
